<a href="https://colab.research.google.com/github/YimingXu1/DS-notes/blob/main/Projects/U.S.%20Patent%20Phrase%20to%20Phrase%20Matching/train_us_patent_deepshare_improve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

Mounted at /content/drive


In [2]:
!pip3 install datasets
!pip3 install transformers
!pip3 install sentencepiece
!pip3 install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 14.5 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 140 kB 93.8 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 75.3 MB/s 
     |████████████████████████████████| 212 kB 94.2 MB/s 
     |████████████████████████████████| 596 kB 78.2 MB/s 
     |████████████████████████████████| 127 kB 99.2 MB/s 
     |████████████████████████████████| 271 kB 97.2 MB/s 
     |████████████████████████████████| 144 kB 89.4 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
    

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
import shutil
import time
import gc
import random
import math
import torch
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorForWholeWordMask
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
from torch import nn
from torch.optim import Adam, SGD, AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
os.environ["WANDB_DISABLED"] = "true"

# Config

In [4]:
class CFG:
    input_path ='./drive/MyDrive/Kaggle/USPatent/uspatent/'
    model_path = 'microsoft/deberta-v3-small' #  nghuyong/ernie-2.0-large-en studio-ousia/luke-large
    scheduler = 'cosine'  # ['linear', 'cosine']
    batch_scheduler = True
    num_cycles = 0.5  # 1.5
    num_warmup_steps = 0.1
    max_input_length = 140
    epochs = 2  # 5
    min_lr = 0.5e-6
    eps = 1e-6
    betas = (0.9, 0.999)
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 5
    batch_size = 100
    seed = 6001
    OUTPUT_DIR = './drive/MyDrive/Kaggle/USPatent/train_checkpoint'
    num_workers = 2
    device='cuda'
    print_freq = 100

## logger

In [5]:
def get_logger(filename=CFG.OUTPUT_DIR+ 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()
LOGGER.info('===============lr_{}==============='.format(CFG.learning_rate))
LOGGER.info('===============seed_{}==============='.format(CFG.seed))
LOGGER.info('===============total_epochs_{}==============='.format(CFG.epochs))
LOGGER.info('===============num_warmup_steps_{}==============='.format(CFG.num_warmup_steps))

===============lr_2e-05===============
===============seed_6001===============
===============total_epochs_2===============
===============num_warmup_steps_0.1===============


# Preproc

In [6]:
train_df = pd.read_csv("./drive/MyDrive/Kaggle/folds-dump-the-two-paths-fix/train_folds_5.csv")

In [7]:
train_df

,id,anchor,target,context,score,context_text,context_text_fix,text,text_fix,score_map,fold
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]abatement of pollution[SEP]human...,abatement[SEP]abatement of pollution[SEP]human...,2,1
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]act of abating[SEP]human necessi...,abatement[SEP]act of abating[SEP]human necessi...,3,3
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]active catalyst[SEP]human necess...,abatement[SEP]active catalyst[SEP]human necess...,1,2
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]eliminating process[SEP]human ne...,abatement[SEP]eliminating process[SEP]human ne...,2,3
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]forest region[SEP]human necessit...,abatement[SEP]forest region[SEP]human necessit...,0,0
...,...,...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden article[SEP]performing...,wood article[SEP]wooden article[SEP]performing...,4,3
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden box[SEP]performing ope...,wood article[SEP]wooden box[SEP]performing ope...,2,2
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden handle[SEP]performing ...,wood article[SEP]wooden handle[SEP]performing ...,2,0
36471,756ec035e694722b,wood article,wooden material,B44,0.75,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden material[SEP]performin...,wood article[SEP]wooden material[SEP]performin...,3,0


# Tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Dataset

In [9]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.texts = df['text_fix'].values.astype(str)
        
        self.label = df['score'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.label[item]
        
        
        inputs = tokenizer(text,
                    max_length=CFG.max_input_length,
                    padding='max_length',
                    truncation=True )
        return {**inputs,
               'labels':torch.as_tensor(label, dtype=torch.float) }


## Model

In [10]:
class Custom_Bert_Simple(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(CFG.model_path)
        self.base = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
        dim = config.hidden_size
        self.dropout = nn.Dropout(p=0)
        self.cls = nn.Linear(dim,1)
        
    def forward(self, input_ids, token_type_ids ,attention_mask, labels=None):
        base_output = self.base(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids
                               )

        output = base_output[0]
        if labels is None:
            return output
        
        else:
            return (nn.MSELoss()(torch.squeeze(output,1),labels), output)

In [11]:
#model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
model = Custom_Bert_Simple()

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

## Trainer

In [12]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # forward pass
        outputs = model(**inputs)
        loss, logits = outputs
        return (loss, outputs) if return_outputs else loss

# Train

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [14]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(CFG.seed)

In [15]:
oof_df = pd.DataFrame()
for fold in range(CFG.num_fold):
    
    tr_data = train_df[train_df['fold']!=fold].reset_index(drop=True)
    va_data = train_df[train_df['fold']==fold].reset_index(drop=True)
    tr_dataset = TrainDataset(tr_data)
    va_dataset = TrainDataset(va_data)
    args = TrainingArguments(
        CFG.OUTPUT_DIR,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=CFG.learning_rate,
        per_device_train_batch_size=CFG.batch_size,
        per_device_eval_batch_size=CFG.batch_size,
        num_train_epochs=CFG.epochs,
        weight_decay=CFG.weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="pearson",
        save_total_limit=1
    )
    model = Custom_Bert_Simple()
    trainer = CustomTrainer(
        model,
        args,
        train_dataset=tr_dataset,
        eval_dataset=va_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    trainer.train()
    
    outputs = trainer.predict(va_dataset)
    predictions = outputs.predictions.reshape(-1)
    va_data['preds'] = predictions
    torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                       './' + "{}_best{}.pth".format(CFG.model_path.replace('/', '_'),fold))
    shutil.rmtree(CFG.OUTPUT_DIR)
    oof_df = pd.concat([oof_df, va_data])

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NO

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.023818,0.804428
2,0.035900,0.023937,0.817083


***** Running Evaluation *****
  Num examples = 7295
  Batch size = 100
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 7295
  Batch size = 100
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584/special_tokens_map.json


Training completed. Do not forget to share your model on hugging

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.028567,0.773888
2,0.054100,0.028946,0.796335


...

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.031460,0.766408
2,0.052600,0.030199,0.787330


***** Running Evaluation *****
  Num examples = 7294
  Batch size = 100
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 7294
  Batch size = 100
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584/special_tokens_map.json


Training completed. Do not forget to share your model on hugging

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.032860,0.773599
2,0.053800,0.029215,0.792659


***** Running Evaluation *****
  Num examples = 7294
  Batch size = 100
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-292/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 7294
  Batch size = 100
Saving model checkpoint to ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Kaggle/USPatent/train_checkpoint/checkpoint-584/special_tokens_map.json


Training completed. Do not forget to share your model on hugging

In [16]:
predictions = oof_df['preds'].values
label = oof_df['score'].values
eval_pred = predictions, label
compute_metrics(eval_pred)

{'pearson': 0.7965931234758126}

In [17]:
oof_df.to_csv('oof_df.csv')